In [1]:
import datasets.utils as datautils
import utils.dgreg_utils as utils
from tensorboardX import SummaryWriter
import torch
from torch import optim
import torch.nn.functional as F
import torch.nn as nn
import argparse
import csv
import os
import numpy as np
from copy import deepcopy
import random
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import torch.utils.data as data
from PIL import Image
from torchvision.transforms import Compose, ToTensor, CenterCrop, RandomCrop, Normalize, Resize
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader


from models.networks import Reg_Domain

In [2]:
net = Reg_Domain(do_emb_size=256, eg_emb_size=16, pretrain=True)
print(net)

/mnt/disk1/mbbank/miniconda3/envs/gpu/lib/python3.8/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(
/mnt/disk1/mbbank/miniconda3/envs/gpu/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/mnt/disk1/mbbank/miniconda3/envs/gpu/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Reg_Domain(
  (extractor): ResnetFeatureExtractor(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu):

In [3]:
model_name = "/mnt/disk1/mbbank/cuong/GraphIQA/ckpt/_step_29.pth"
pretained_model = torch.load(model_name, map_location='cuda:1')
model_dict = net.state_dict()
state_dict = {k: v for k, v in pretained_model.items() if k in model_dict.keys()}
model_dict.update(state_dict)
net.load_state_dict(model_dict)

<All keys matched successfully>

In [4]:
data_train = '../Data/train/'
data_test = '../Data/test/'
data_val = '../Data/valid/'

In [5]:
import torch.utils.data as data

class CustomDataSet(data.Dataset):

    def __init__(self, root,  patch_num):
        self.root = root
        self.samples = []
        self.label = []
        for i in range(5):
            self.samples.extend([root + str(i) + '/' + path_image for path_image in os.listdir(root + '/' + str(i))])
            self.label.extend([i for j in range(len(os.listdir(root + '/' +str(i))))])
        self.transform = self.img_transform(224)

    def __getitem__(self, index):
        target = self.label[index]
        sample = Image.open(self.samples[index])
        sample.convert('RGB')
        sample = self.transform(sample)
        return sample, target

    def __len__(self):
        length = len(self.samples)
        return length

    def img_transform(self, crop_size):
        normalize = Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        return Compose([
            Resize([224,224]),
#             RandomCrop(crop_size),
            ToTensor(),
#             normalize
            ])

In [6]:
data = CustomDataSet(data_test, patch_num=25)
data.__getitem__(17204)

(tensor([[[0.3216, 0.3255, 0.3294,  ..., 0.1608, 0.1608, 0.1569],
          [0.2902, 0.2941, 0.3020,  ..., 0.1529, 0.1569, 0.1569],
          [0.2510, 0.2588, 0.2745,  ..., 0.1686, 0.1725, 0.1686],
          ...,
          [0.6863, 0.6824, 0.4902,  ..., 0.0157, 0.0196, 0.0157],
          [0.6824, 0.6824, 0.4941,  ..., 0.0118, 0.0118, 0.0118],
          [0.6784, 0.6784, 0.5098,  ..., 0.0118, 0.0157, 0.0118]],
 
         [[0.2706, 0.2745, 0.2745,  ..., 0.1804, 0.1804, 0.1765],
          [0.2392, 0.2431, 0.2471,  ..., 0.1765, 0.1804, 0.1804],
          [0.2000, 0.2078, 0.2235,  ..., 0.1922, 0.1961, 0.1961],
          ...,
          [0.6549, 0.6471, 0.4471,  ..., 0.0157, 0.0196, 0.0157],
          [0.6510, 0.6471, 0.4549,  ..., 0.0118, 0.0118, 0.0118],
          [0.6510, 0.6471, 0.4745,  ..., 0.0118, 0.0157, 0.0118]],
 
         [[0.1451, 0.1490, 0.1569,  ..., 0.2275, 0.2275, 0.2235],
          [0.1137, 0.1176, 0.1294,  ..., 0.2235, 0.2275, 0.2314],
          [0.0745, 0.0824, 0.0980,  ...,

In [7]:
data.__len__()

17205

In [8]:
class DataModule(pl.LightningDataModule):

    def setup(self, stage):        
        self.data_train = CustomDataSet(root=data_train, patch_num=25)
        self.data_val = CustomDataSet(root=data_val, patch_num=25)
        self.data_test = CustomDataSet(root=data_test, patch_num=25)
        self.batch_size = 10
        
    def test_dataloader(self):
        return DataLoader(self.data_test, batch_size=self.batch_size, num_workers=8, pin_memory=True)
                                                
    def train_dataloader(self):
        return DataLoader(self.data_train, batch_size=self.batch_size, num_workers=8, pin_memory=True, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.data_val, batch_size=self.batch_size, num_workers=8, pin_memory=True)


In [9]:
from models.networks import Reg_Domain
import math

class model_qa(pl.LightningModule):
    def __init__(self):
        super(model_qa,self).__init__()
        self.base_model = net
        
    def MSEloss(self, outputs, labels):
        return torch.nn.MSELoss()(outputs.to(torch.float32), labels.to(torch.float32))
    
    def forward(self,x):
        x = self.base_model(x)
        return x    
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x = x.float()
        outputs, _, _, _ = self.forward(x)
        loss = self.MSEloss(outputs.to(torch.float32), y)
        self.log('train_loss', loss)
        return loss 
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        x = x.float()
        outputs, _, _, _ = self.forward(x)
        loss = self.MSEloss(outputs.to(torch.float32), y)
        loss_mae = nn.L1Loss()(outputs.to(torch.float32), y)
        self.log('val_loss', loss)
        return {'mse': loss, 'mae': loss_mae}
    
    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        x = x.float()
        outputs, _, _, _ = self.forward(x)
        loss = self.MSEloss(outputs.to(torch.float32), y)
        loss_mae = nn.L1Loss()(outputs.to(torch.float32), y)
        self.log('test_loss', loss)
        return {'mse': loss, 'mae': loss_mae}
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat, _, _, _ = self.base_model(x)
        return y_hat
    
    def training_epoch_end(self, training_step_outputs):
        loss = torch.stack([f['loss'] for f in training_step_outputs]).mean()
        print('training loss', loss)
    
    def validation_epoch_end(self, validation_step_outputs):
        loss_mse = torch.stack([f['mse'] for f in validation_step_outputs]).mean()
        loss_mae = torch.stack([f['mae'] for f in validation_step_outputs]).mean()
        print('val loss mse', loss_mse)
        print('val loss mae', loss_mae)
        
    def test_epoch_end(self, test_step_outputs):
#         for dataloader_outputs in test_step_outputs:
#             print(dataloader_outputs)
        loss_mse = torch.stack([f['mse'] for f in test_step_outputs]).mean()
        loss_mae = torch.stack([f['mae'] for f in test_step_outputs]).mean()
        print('test loss mse', loss_mse)
        print('test loss mae', loss_mae)
        
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-2)
        return optimizer

In [10]:
model = model_qa()
model

model_qa(
  (base_model): Reg_Domain(
    (extractor): ResnetFeatureExtractor(
      (feature_extractor): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, af

In [11]:
data_module = DataModule()
model = model_qa()

trainer = pl.Trainer(accelerator="gpu", devices=[1], max_epochs=10)
trainer.fit(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name       | Type       | Params
------------------------------------------
0 | base_model | Reg_Domain | 51.5 M
------------------------------------------
28.0 M    Trainable params
23.5 M    Non-trainable params
51.5 M    Total params
206.085   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

val loss mse tensor(1519.7832, device='cuda:1')
val loss mae tensor(35.8287, device='cuda:1')


/mnt/disk1/mbbank/miniconda3/envs/gpu/lib/python3.8/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/mnt/disk1/mbbank/miniconda3/envs/gpu/lib/python3.8/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

val loss mse tensor(0.9062, device='cuda:1')
val loss mae tensor(0.7438, device='cuda:1')
training loss tensor(0.9242, device='cuda:1')


Validation: 0it [00:00, ?it/s]

val loss mse tensor(0.9056, device='cuda:1')
val loss mae tensor(0.7460, device='cuda:1')
training loss tensor(0.9243, device='cuda:1')


`Trainer.fit` stopped: `max_epochs=10` reached.


In [12]:
data_module = DataModule()
trainer.test(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

/mnt/disk1/mbbank/miniconda3/envs/gpu/lib/python3.8/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/mnt/disk1/mbbank/miniconda3/envs/gpu/lib/python3.8/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


test loss mse tensor(0.8637, device='cuda:1')
test loss mae tensor(0.7248, device='cuda:1')
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.8625409603118896
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.8625409603118896}]